### ZONAL STATISTICS AND MERGE ALL DATA FROM NOTEBOOKS 1 THRU 4

In [8]:
# Modules.
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
from rasterstats import zonal_stats
import rasterio

In [9]:
# Paths.
tracts_path = Path("data/nyc_tracts_2020/nyc_tracts_2020.shp")

# ndvi_path = Path("data/raster/processed/ndvi_summer_2025.tif")
# ndwi_path = Path("data/raster/processed/ndwi_summer_2025.tif")
# ndbi_path = Path("data/raster/processed/ndbi_summer_2025.tif")

tree_canopy_path = Path("data/raster/nlcd_raster/nlcd_tree_canopy_2023.tiff")
impervious_path = Path("data/raster/nlcd_raster/nyc_ncld_impervious_2024.tiff")
landcover_path = Path("data/raster/nlcd_raster/nyc_ncld_land_cover_2024.tiff")

acs_path = Path("data/acs/acs_socioeconomic_tract_2022.csv")
jfk_heat_path = Path("data/heat/JFK_2025_JJA_extreme_heat_90.csv")
calls_path = Path("data/nyc_311/panel/nyc_311_tract_day_2025.csv")

output_path = Path("data/model/nyc_tract_summer_2025_final.csv")

tracts = gpd.read_file(tracts_path)
tracts = tracts.to_crs(4326)
tracts.head()

,ctlabel,borocode,boroname,ct2020,boroct2020,cdeligibil,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry
0,1,1,Manhattan,000100,1000100,I,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,10833.043929,1.843005e+06,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,14.01,1,Manhattan,001401,1001401,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((-73.98837 40.71645, -73.98754 40.716..."
2,14.02,1,Manhattan,001402,1001402,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,"POLYGON ((-73.98507 40.71908, -73.98423 40.718..."
3,18,1,Manhattan,001800,1001800,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001800,6391.921174,2.399277e+06,"POLYGON ((-73.98985 40.72052, -73.98972 40.720..."
4,22.01,1,Manhattan,002201,1002201,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061002201,5779.062607,1.740174e+06,"POLYGON ((-73.97875 40.71993, -73.97879 40.719..."


In [10]:
# Zonal statistics helper function.
def zonal_mean(file_path, target_gdf):
    """
    Computes mean zonal statistics for a raster over polygons.
    Automatically reprojects polygons to raster CRS.
    """    
    with rasterio.open(file_path) as src:
        raster_crs = src.crs

    gdf_proj = target_gdf.to_crs(raster_crs)

    zonal_statistic = zonal_stats(
        gdf_proj,
        file_path,
        stats = ["mean"],
        nodata = np.nan,
        geojson_out = False
    )
    
    return [x["mean"] for x in zonal_statistic]

In [11]:
# NLCD calculations with print checks, because I learned my lesson.
print("Computing tree canopy percent zonal stats.")
tracts["tree_canopy_pct"] = zonal_mean(tree_canopy_path, tracts)

print("Computing impervious surface percent zonal stats.")
tracts["impervious_pct"] = zonal_mean(impervious_path, tracts)

print("Computing land cover zonal stats.")
tracts["landcover_mean"] = zonal_mean(landcover_path, tracts)

print("NLCD zonal stats complete.")

Computing tree canopy percent zonal stats.
Computing impervious surface percent zonal stats.
Computing land cover zonal stats.
NLCD zonal stats complete.


In [12]:
# Landsat calculations with print checks, because I learned my lesson.
# print("Computing Landsat NDVI zonal stats.")
# tracts["ndvi_mean"] = zonal_mean(ndvi_path, tracts)

# print("Computing Landsat NDWI zonal stats.")
# tracts["ndwi_mean"] = zonal_mean(ndwi_path, tracts)

# print("Computing Landsat NDBI zonal stats.")
# tracts["ndbi_mean"] = zonal_mean(ndbi_path, tracts)

# print("Finished Landsat zonal statistics.")

In [13]:
# ACS.
acs = pd.read_csv(acs_path, dtype = {"GEOID":"string"})
acs.columns = acs.columns.str.upper()
acs.head()

,GEOID,TOTAL_POP,MEDIAN_INCOME,NO_VEHICLE_HH,HH_TOTAL,POP_DENSITY,POVERTY_RATE,PCT_BACHELORS_PLUS,PCT_RENTERS,PCT_LIMITED_ENGLISH
0,36081045000,2004,137109,35,761,0.001056,0.069860,0.406198,0.540079,0.000000
1,36081045400,4793,65610,306,1696,0.001920,0.188608,0.448410,0.592571,0.000000
2,36081045500,13869,75033,0,4482,0.008659,0.211639,0.261328,1.000000,0.001110
3,36081045600,1244,75500,22,408,0.000710,0.091640,0.434879,0.098039,0.000000
4,36081044602,5210,44700,45,1550,0.002835,0.304574,0.270916,0.856774,0.012887


In [14]:
# Heat.
heat = pd.read_csv(jfk_heat_path, parse_dates = ["DATE"])
heat.columns = heat.columns.str.upper()
heat.head()

,DATE,TMAX_F,EXTREME_HEAT
0,2025-06-01,73.0,no
1,2025-06-02,72.0,no
2,2025-06-03,75.9,no
3,2025-06-04,75.9,no
4,2025-06-05,81.0,no


In [15]:
# 311.
calls = pd.read_csv(calls_path, dtype = {"GEOID":"string"}, parse_dates = ["date"])
calls.columns = calls.columns.str.upper()
calls.head()

,GEOID,DATE,TOTAL_CALLS,QOL_CALLS,QOL_PCT
0,36005000100,2025-06-30,1,1,1.000000
1,36005000100,2025-07-23,1,0,0.000000
2,36005000100,2025-08-04,1,1,1.000000
3,36005000200,2025-06-01,8,6,0.750000
4,36005000200,2025-06-02,3,1,0.333333


In [16]:
# Merge ACS.
tracts.columns = tracts.columns.str.upper()
tracts_df = tracts.drop(columns = "GEOMETRY").copy()

merged = tracts_df.merge(acs, on = "GEOID", how = "left")
print("After ACS merge:", merged.shape)

After ACS merge: (2325, 25)


In [17]:
# Drop tracts with no ACS population.
merged = merged[merged["TOTAL_POP"].notna()].copy()

print("Tracts after removing non-ACS tracts:", merged["GEOID"].nunique())

Tracts after removing non-ACS tracts: 2229


In [18]:
# Add constant merge key.
merged["KEY"] = 1
heat["KEY"] = 1

# Merge heat.
tract_heat = merged.merge(heat, on = "KEY").drop(columns = ["KEY"])
print("Tract by day:", tract_heat.shape)

Tract by day: (196152, 28)


In [19]:
# Final merge.
final = tract_heat.merge(
    calls,
    on = ["GEOID", "DATE"],
    how = "left"
)

print("After merging 311:", final.shape)

After merging 311: (196152, 31)


In [20]:
# Take care of NA values.
final["TOTAL_CALLS"] = final["TOTAL_CALLS"].fillna(0)
final["QOL_CALLS"] = final["QOL_CALLS"].fillna(0)

final["LOG_TOTAL_CALLS"] = np.log(final["TOTAL_CALLS"].replace(0, 1))

In [21]:
# Final data.
final.to_csv(output_path, index = False)

print(f"Final data for modeling: {output_path}")

Final data for modeling: data\model\nyc_tract_summer_2025_final.csv


In [24]:
# Write final data for high heat and normal heat days.
high_heat = final[final["EXTREME_HEAT"] == 'yes']
normal_heat = final[final["EXTREME_HEAT"] == 'no']

high_heat.to_csv("data/model/model_high_heat.csv", index = False)
normal_heat.to_csv("data/model/model_normal_heat.csv", index = False)

print("Model data:")
print(" - model_high_heat.csv")
print(" - model_normal_heat.csv")

Model data:
 - model_high_heat.csv
 - model_normal_heat.csv
